## Implementation of a simple RAG system

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from pathlib import Path
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_core.runnables import RunnableLambda
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import os
from dotenv import load_dotenv


In [2]:
# Load PDFs
pdf_folder = Path("pdfs/")

all_docs = []
for pdf_file in pdf_folder.glob("*.pdf"):
    loader = PyPDFLoader(str(pdf_file))
    all_docs.extend(loader.load())

Ignoring wrong pointing object 37 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 37 0 (offset 0)


In [3]:
# Chunk the data using recursive chunking
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50
)

chunks = splitter.split_documents(all_docs)

In [4]:
# Load embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [5]:
# Store the embeddings locally in ChromaDB

vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

vectordb.persist()

/Users/shraddharamesh/Documents/Artificial Intelligence/GitHub Projects/outskill-ai-practice/rag-langchain/rag/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/var/folders/0d/_4fqv2g50f36w1vmz28wqkyr0000gn/T/ipykernel_46303/3190493980.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [9]:
# Load model and tokenizer
model_name = "google/flan-t5-small"  # Or try flan-t5-base if M2 can handle it

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Create text2text pipeline (note: temperature may be ignored)
hf_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    device=-1
)

# Wrap in LangChain HuggingFace LLM (new path!)
hf_llm = HuggingFacePipeline(pipeline=hf_pipeline)

Device set to use cpu


In [10]:
retriever = vectordb.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=hf_llm,
    retriever=retriever,
    chain_type="stuff"
)

In [11]:
response = hf_llm.invoke("What is ornithology?")
print(response)

ornithology ornithology


In [12]:
query = "What is ornithology?"

# ✅ RAG response (using LangChain's new `.invoke()` method)
response_rag = qa_chain.invoke(query)

# ✅ LLM-only response
ragless_prompt = f"Answer the following question:\n{query}"
response_llm_only = hf_llm.invoke(ragless_prompt)

# 📊 Display the results
print("🔹 LLM-Only Response:\n", response_llm_only)
print("\n" + "="*80 + "\n")
print("🔸 RAG-Augmented Response:\n", response_rag)

/Users/shraddharamesh/Documents/Artificial Intelligence/GitHub Projects/outskill-ai-practice/rag-langchain/rag/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Token indices sequence length is longer than the specified maximum sequence length for this model (541 > 512). Running this sequence through the model will result in indexing errors


🔹 LLM-Only Response:
 Ornithology


🔸 RAG-Augmented Response:
 {'query': 'What is ornithology?', 'result': 'Ornithology is a branch of science devoted to the study of birds. Ornithology encompasses all types of research involving birds, such as habitat studies and migration patterns. Ornithology encompasses all types of research involving birds, such as habitat studies and migration patterns. Ornithology encompasses all types of research involving birds, such as habitat studies and migration patterns. Ornithology encompasses all types of research involving birds, such as habitat studies and migration patterns. Ornithology encompasses all types of research involving birds, such as habitat studies and migration patterns. Ornithology encompasses all types of research involving birds, such as habitat studies and migration patterns. Ornithology encompasses all types of research involving birds, such as habitat studies and migration patterns. Ornithology encompasses all types of research inv

### Test with Open AI LLM

In [6]:
load_dotenv()  # take environment variables from .env.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set. Please set it in your .env file.")

In [7]:
# A prompt template with RAG context
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template='''
    Given the context below, answer the question to the best of your ability.
    Be polite and helpful.
    If the context doesn't contain the answer, say "I don't know" and offer to help with something else.

    Context: {context}

    Question: {question}

    Answer:
    '''
)


In [ ]:


# ✅ LLM Setup
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# ✅ Retrieval setup (RAG)
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_template}
)
# ✅ Query to compare
query = "What are the main functions of bird feathers?"

# 🔸 1. LLM-Only (No RAG)
llm_only_prompt = f"Answer the following question clearly and concisely:\n\n{query}"
llm_only_response = llm.invoke(llm_only_prompt)

# 🔹 2. RAG-Based (uses retrieved context)
rag_response = rag_chain({"query": query})

# 🖥️ Print Results
print("🔸 LLM-Only Response:\n", llm_only_response.content)
print("\n" + "="*80 + "\n")
print("🔹 RAG-Augmented Response:\n", rag_response['result'])
print("\nSources:")
    for doc in rag_response["source_documents"]:
        print(f"- {doc.metadata.get('source', 'Unknown source')}, page {doc.metadata.get('page', 'N/A')}")


/var/folders/0d/_4fqv2g50f36w1vmz28wqkyr0000gn/T/ipykernel_46303/2701949801.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
/var/folders/0d/_4fqv2g50f36w1vmz28wqkyr0000gn/T/ipykernel_46303/2701949801.py:20: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  rag_response = rag_chain({"query": query})
/Users/shraddharamesh/Documents/Artificial Intelligence/GitHub Projects/outskill-ai-practice/rag-langchain/rag/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and wi

🔸 LLM-Only Response:
 The main functions of bird feathers are insulation, flight, protection, and display.


🔹 RAG-Augmented Response:
 The main functions of bird feathers include insulation, waterproofing, flight, communication, and camouflage.


In [9]:
# ✅ Query to compare
query = "Why are songbirds so diverse? Be brief."

# 🔸 1. LLM-Only (No RAG)
llm_only_prompt = f"Answer the following question clearly and concisely:\n\n{query}"
llm_only_response = llm.invoke(llm_only_prompt)

# 🔹 2. RAG-Based (uses retrieved context)
rag_response = rag_chain({"query": query})

# 🖥️ Print Results
print("🔸 LLM-Only Response:\n", llm_only_response.content)
print("\n" + "="*80 + "\n")
print("🔹 RAG-Augmented Response:\n", rag_response['result'])

/Users/shraddharamesh/Documents/Artificial Intelligence/GitHub Projects/outskill-ai-practice/rag-langchain/rag/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


🔸 LLM-Only Response:
 Songbirds are diverse due to a combination of factors such as their ability to adapt to various habitats, their wide range of diets, and their complex vocalizations for communication and mating purposes.


🔹 RAG-Augmented Response:
 Songbirds are diverse due to their ability to learn songs and other mate-attracting or territory-protecting behaviors, which are potential key innovations that drive their diversity.


In [10]:
# ✅ Query to compare
query = "How are bird bones similar and different to that of humans? Answer in a tabular form"

# 🔸 1. LLM-Only (No RAG)
llm_only_prompt = f"Answer the following question clearly and concisely:\n\n{query}"
llm_only_response = llm.invoke(llm_only_prompt)

# 🔹 2. RAG-Based (uses retrieved context)
rag_response = rag_chain({"query": query})

# 🖥️ Print Results
print("🔸 LLM-Only Response:\n", llm_only_response.content)
print("\n" + "="*80 + "\n")
print("🔹 RAG-Augmented Response:\n", rag_response['result'])

/Users/shraddharamesh/Documents/Artificial Intelligence/GitHub Projects/outskill-ai-practice/rag-langchain/rag/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


🔸 LLM-Only Response:
 | Aspect          | Bird Bones                  | Human Bones                 |
|-----------------|-----------------------------|-----------------------------|
| Structure       | Hollow and lightweight      | Dense and solid             |
| Composition     | Mostly made of calcium      | Mostly made of calcium      |
| Function        | Aid in flight and balance   | Support body weight and movement |
| Number          | More numerous and smaller   | Fewer and larger            |
| Growth          | Can repair and remodel quickly | Slower repair and remodeling |
| Strength        | Strong but more fragile     | Strong and less fragile     |


🔹 RAG-Augmented Response:
 | Aspect          | Bird Bones                               | Human Bones                              |
|-----------------|------------------------------------------|------------------------------------------|
| Structure       | Highly specialized for flying            | Not specialized for flyin